Euclid GC Fisher Matrix Code

Authors: Alkistis Pourtsidou and Dida Markovic, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [ ]:
%matplotlib inline

# Set up all kinds of libraries
import sys, platform, os
from matplotlib import pyplot as plt
from glob import glob

# Set up science stuff
import numpy as np
import scipy
from scipy.interpolate import interp1d
from __future__ import division
from scipy import integrate, linalg
pi=np.pi

# Import custom modules in this folder
import misc

# Importing CAMB
print('Using CAMB installed at '+ os.path.realpath(os.path.join(os.getcwd(),'..')))
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import camb
from camb import model, initialpower

In [ ]:
# Options for running this notebook (should go into a config file, really)

### Flags

# Flag to read from files
fromfile = True

# Type of interpolation
interp_type = 'cubic'

# Derivative approach
nderiv = True

# Path flag to the files to be read
SlavMonkey = False
if SlavMonkey:
    path="/Users/dida/Google Drive/Euclid/IST GC/Inputs/"
else:
    path="/Users/alkistis/Google Drive/IST GC/Inputs/"
pathrt = path + "IST_Pks8sqRatios_plmn_z/"
pathfg = path
pathPk = path + "IST_Pk_plmn_z/"


### Fiducial cosmological parameters

hubble=0.67
omegab=0.022445/hubble**2
omegac=0.121203/hubble**2
om0=omegac+omegab
H00=100*hubble
Ass=2.1265e-9
nss=0.96

gamma=0.545

c=3e5


### Set up Euclid

Area=15000.0 #deg^2
omegatot = Area*pow(pi/180,2)
Dzbin = 0.1
sig_p = 0.001


### Set up the redshift binned functions from IST docs

zlist = np.arange(0.7,2.1,0.1)
ztest = zlist[0]
Nzbins = len(zlist)

biaslist = [1.083, 1.125, 1.104, 1.126, 1.208, 1.243, 1.282, 1.292, 1.363, 1.497, 1.486, \
            1.491, 1.573, 1.568]

kmin = 0.001
kmax = 0.2

dn3 = [2434.28, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, \
       2308.975, 1541.831, 1474.707, 893.716, 497.613]

print zlist
print len(zlist), len(dn3)


### Set up the Fisher matrix calculation

params = ["0:lnH","1:lnDA","2:lnfsig8","3:lnbsig8","4:Ps","5:ns","6:wb","7:wm","8:h"]
shape_params = [5,6,7,8]

In [ ]:
#generic reading function
def getfile(rootname,zc,zlist=zlist):
    allfiles = glob(rootname)
    if len(allfiles)==0:raise Exception('no files found in '+rootname)
    for fn in allfiles:
        index = int(fn.split('=')[-1][:2])
        if np.abs(zc-zlist[index])<1e-5:
            return fn
    raise Exception('redshift ' + str(zc) + ' not found')
    
getfile(pathPk+'/Pk_ist_fid_iz=*.dat',0.8)

In [ ]:
# Get the power spectrum from CAMB if not reading from files
if not fromfile:
    
    # Set up the fiducial cosmology for CAMB
    pars = camb.CAMBparams()
    
    # Set cosmology
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    
    # Calculate results for these parameters
    results = camb.get_results(pars)
    
    # Get matter power spectrum at z=0: P(k,z=0)

    # Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)

    # Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)

In [ ]:
#VALIDATION

#Get various background functions and derived parameters
#pars = camb.CAMBparams()
#pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
#pars.set_dark_energy() #LCDM (default)
#pars.InitPower.set_params(ns=nss, r=0, As=Ass)
#pars.set_for_lmax(2500, lens_potential_accuracy=0);

# Calculate results for these parameters
#results = camb.get_background(pars)
#print 'Derived parameter dictionary:\n', results.get_derived_params()

In [ ]:
#for zz in zlist:
#    chi = results.comoving_radial_distance(zz)
#    DA = results.angular_diameter_distance(zz)
#    Hz = results.hubble_parameter(zz)
#    print round(zz,4), round(Hz,4), round(chi,4), round(DA,4)

In [ ]:
# Geometry functions for a spatially flat Universe

# Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00

# Define the cosmological distances
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)


In [ ]:
# LCDM growth rate and growth factor
if not fromfile:

    def fg(zz):
        omz=om0*pow(1+zz,3)/pow(Ez(zz),2)
        return pow(omz,gamma)

    def Dg_dz(zz):
        return -fg(zz)/(1+zz)

    def Dgz(zc):
        start_z = 0.0
        ans = scipy.integrate.romberg(Dg_dz, start_z, zc)
        return np.exp(ans)
    
else:

    growthfilename = pathfg+"/IST_zetha_fg_growthfactor.dat"
    datafg = np.genfromtxt(growthfilename)
    
    zg = datafg[:,0]
    fg = datafg[:,1]
    Dg = datafg[:,2]
    
    def fg(zz, zi=zg, fi=fg):
        ans = interp1d(zi, fi)
        return ans(zz)

    def Dgz(zz, zi=zg, di=Dg):
        ans = interp1d(zi, di)
        return ans(zz)
    
print fg(0.7), fg(1.7)

In [ ]:
if fromfile:
    
    def ratio(zc,pmf,par=''):
        """ par - string of the shape parameter index:
                  00 = wm
                  01 = wb
                  02 = h
                  03 = ns
            pmf - string that is mn, fid or pl
        """
        
        if pmf is 'fid':
            fileroot = pathrt + 'Pks8sqRatio_ist_' + pmf + '_iz=*.dat'
        else:
            fileroot = pathrt + 'Pks8sqRatio_ist_' + pmf + '_ip=' + par + '_iz=*.dat'
        
        filename = getfile(fileroot, zc)
            
        data = np.genfromtxt(filename, skip_header=3)
        kh = data[:,0]
        rat = data[:,1] 
        
        return interp1d(kh, rat, kind=interp_type)
    
    
    def sig8(zc):
        """ par - string of the shape parameter index:
                  00 = wm
                  01 = wb
                  02 = h
                  03 = ns
        """
        fileroot = pathrt + 'Pks8sqRatio_ist_fid_iz=*.dat'
        filename = getfile(fileroot, zc)
        
        data = np.genfromtxt(filename, skip_header=3)
        return data[0,2]
    
    print sig8(2.0)

In [ ]:
# Get the P(k) and sig8 at z=0

# Construct P(k,z=0) interpolating function and get fiducial σ8 (at z=0)
if not fromfile:
    
    kh, _z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)
    
    sig8_fid = results.get_sigma8()[0]
    print sig8_fid # 0.830287480109

    rt0_fid = interp1d(kh, pk[0]/pow(sig8_fid,2))
    
    Pkz0 = interp1d(kh, pk[0])
    print Pkz0(0.1) # 5704.51244798
    
    def Pkz(kk,zc):
        return pow(Dgz(zc),2)*Pkz0(kk)  
else:
    
    filename = getfile(pathrt+"/Pks8sqRatio_ist_fid_iz=*.dat",0.7)

    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rat = data[:,1] 
    rt0_fid = interp1d(kh, rat, kind=interp_type)
    
    
    fileroot = pathPk+"/Pk_ist_fid_iz=*.dat"    
    def Pkz(kk,zc):
        
        filename = getfile(fileroot,zc)
        
        data = np.genfromtxt(filename, skip_header=3)
        kh = data[:,0]
        pk = data[:,1] 
        
        return interp1d(kh, pk, kind=interp_type)(kk)

# Construct the observed power spectrum, P_gg(k,μ,z)
def Pgg(kk,mu,zc,bgs8=None,fgs8=None):
    s8 = sig8(zc)
    if bgs8 is None:
        bgs8 = bg * s8
    if fgs8 is None:
        fgs8 = fg(zc) * s8
    return pow(bgs8+fgs8*mu**2,2)*Pkz(kk,zc)/s8**2

print Pkz(0.1,0.8)#2497.21546467
print Pgg(0.1,0.,0.8,bgs8=sig8(0.8))

In [ ]:
# Calculate the survey properties

# Redshift errors

def photoz(kk,mu,zc):
    sigz = sig_p*(1+zc) 
    return np.exp(-pow(kk*mu,2)*pow(c*sigz,2)/pow(Hz(zc),2))
#print photoz(0.5,0.5,ztest)

# Survey (bin) volume [Mpc^3]

def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

#testing with recipe formula
#def Vsur_rec(zc):
#    z1 = zc-Dzbin/2
#    z2 = zc+Dzbin/2
#    return Area*(4*pi/3)*((1+z2)**3*DA(z2)**3-(1+z1)**3*DA(z1)**3)/(4*pi*(180/pi)**2)

def Pshot(zc):
    return Vsur(zc)/Ngal

# Effective volume (the hubble**3 factors are for units consistency)

def Veff(kk,mu,zc):
    return hubble**3*Vsur(zc)*(Pgg(kk,mu,zc)*photoz(kk,mu,zc)/(Pgg(kk,mu,zc)*photoz(kk,mu,zc)\
                                                     +hubble**3*Pshot(zc)))**2
#print ztest, "%.4g" %  Vsur(ztest), "%.4g" %  Vsur_rec(ztest)

In [ ]:
# Fisher matrix derivatives 

# z-dependent parameters

#the σ8's cancel out so we don't include them

def dlnP_dPs(kk,mu,zc):
    return 1.0/(Pgg(kk,mu,zc)*photoz(kk,mu,zc))

if not nderiv:
    def dlnP_dlnfsig8(kk,mu,zc):
        return 2*mu**2*fg(zc)/(bg+mu**2*fg(zc))

    def dlnP_dlnbsig8(kk,mu,zc):
        return 2*bg/(bg+mu**2*fg(zc))

    def dlnP_dlnDA(kk,mu,zc):
        dk = (kmax-kmin)/400
        return -2.0+4*mu**2*(1-mu**2)*fg(zc)/(bg+mu**2*fg(zc))\
                -kk*(1-mu**2)*(1/Pkz(kk,zc))*(Pkz(kk+dk,zc)-Pkz(kk-dk,zc))/(2*dk)

    def dlnP_dlnH(kk,mu,zc):
        dk = (kmax-kmin)/400
        return 1.0+4*mu**2*(1-mu**2)*fg(zc)/(bg+mu**2*fg(zc))\
                +kk*mu**2*(1/Pkz(kk,zc))*(Pkz(kk+dk,zc)-Pkz(kk-dk,zc))/(2*dk)

else:
    
    # NUMERICAL DERIVATIVES
    def dlnP_dlnfsig8(kk,mu,zc):
        fs8_fid = fg(zc)*sig8(zc)
        step_fs8 = 0.01
        fs8_p = fs8_fid*(1+step_fs8)
        fs8_m = fs8_fid*(1-step_fs8)

        Pgg_fid = Pgg(kk,mu,zc)
        Pgg_p = Pgg(kk,mu,zc,fgs8=fs8_p)
        Pgg_m = Pgg(kk,mu,zc,fgs8=fs8_m)

        return (fs8_fid/Pgg_fid)*(Pgg_p-Pgg_m)/(2*step_fs8*fs8_fid)

    def dlnP_dlnbsig8(kk,mu,zc):
        bs8_fid = bg*sig8(zc)
        step_bs8 = 0.01
        bs8_p = bs8_fid*(1+step_bs8)
        bs8_m = bs8_fid*(1-step_bs8)

        Pgg_fid = Pgg(kk,mu,zc)
        Pgg_p = Pgg(kk,mu,zc,bgs8=bs8_p)
        Pgg_m = Pgg(kk,mu,zc,bgs8=bs8_m)

        return (bs8_fid/Pgg_fid)*(Pgg_p-Pgg_m)/(2*step_bs8*bs8_fid)

    def dlnP_dlnDA(kk,mu,zc):
        DA_fid = DA(zc)
        step_DA = 0.1
        DA_p = DA_fid*(1+step_DA)
        DA_m = DA_fid*(1-step_DA)

        Pgg_fid = Pgg(kk,mu,zc)

        k_p = kk*(DA_fid/DA_p)*np.sqrt(1+mu**2*((DA_p/DA_fid)**2 - 1)) 
        mu_p = mu*(DA_p/DA_fid)/np.sqrt(1+mu**2*((DA_p/DA_fid)**2 - 1)) 
        Pgg_p = (DA_fid/DA_p)**2 * Pgg(k_p,mu_p,zc)

        k_m = kk*(DA_fid/DA_m)*np.sqrt(1+mu**2*((DA_m/DA_fid)**2 - 1)) 
        mu_m = mu*(DA_m/DA_fid)/np.sqrt(1+mu**2*((DA_m/DA_fid)**2 - 1)) 
        Pgg_m = (DA_fid/DA_m)**2 * Pgg(k_m,mu_m,zc)

        return (DA_fid/Pgg_fid)*(Pgg_p-Pgg_m)/(2*step_DA*DA_fid)

    def dlnP_dlnH(kk,mu,zc):
        H_fid = Hz(zc)
        step_H = 0.1
        H_p = H_fid*(1+step_H)
        H_m = H_fid*(1-step_H)

        Pgg_fid = Pgg(kk,mu,zc)

        k_p = kk*np.sqrt(1+mu**2*((H_p/H_fid)**2 - 1)) 
        mu_p = mu*(H_p/H_fid)/np.sqrt(1+mu**2*((H_p/H_fid)**2 - 1)) 
        Pgg_p = (H_p/H_fid) * Pgg(k_p,mu_p,zc)

        k_m = kk*np.sqrt(1+mu**2*((H_m/H_fid)**2 - 1)) 
        mu_m = mu*(H_m/H_fid)/np.sqrt(1+mu**2*((H_m/H_fid)**2 - 1)) 
        Pgg_m = (H_m/H_fid) * Pgg(k_m,mu_m,zc)

        return (H_fid/Pgg_fid)*(Pgg_p-Pgg_m)/(2*step_H*H_fid)    
    
#bg = 1    
#print dlnP_dlnfsig8(0.01,1.,0.8), dlnP_dlnfsig8_ND(0.01,1.,0.8)
#print dlnP_dlnbsig8(0.01,1.,0.8), dlnP_dlnbsig8_ND(0.01,1.,0.8)
#print dlnP_dlnDA(0.01,1,0.8), dlnP_dlnDA_ND(0.01,1,0.8)
#print dlnP_dlnH(0.01,0.9,0.8), dlnP_dlnH_ND(0.01,0.9,0.8)

In [ ]:
# Shape parameter derivativers from scratch

if not fromfile:
    # Numerical derivative wrt ns 

    step_ns = 0.01*nss
    ns_p = nss+step_ns
    ns_m = nss-step_ns

    #DERIV PLUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=ns_p, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    sig8_ns_p = results.get_sigma8()[0]
    #Pkz0_ns_p = interp1d(kh, pk[0])
    
    rt0_ns_p = interp1d(kh,pk[0]/pow(sig8_ns_p,2))

    #DERIV MINUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=ns_m, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    sig8_ns_m= results.get_sigma8()[0]
    #Pkz0_ns_m = interp1d(kh, pk[0])
    
    rt0_ns_m = interp1d(kh,pk[0]/pow(sig8_ns_m,2))

    #RESET FIDUCIAL
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)

    #print sig8_ns_m, sig8_fid, sig8_ns_p
    #print results.get_sigma8()[0]
    
    # Numerical derivative wrt wb 

    wb = omegab*pow(hubble,2)
    wc = omegac*pow(hubble,2)
    step_wb = 0.01*wb
    wb_p = wb+step_wb
    wb_m = wb-step_wb
    wc_p = wc-step_wb
    wc_m = wc+step_wb

    #DERIV PLUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=wb_p, omch2=wc_p,omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    sig8_wb_p = results.get_sigma8()[0]
    #Pkz0_wb_p = interp1d(kh, pk[0])

    rt0_wb_p = interp1d(kh,pk[0]/pow(sig8_wb_p,2))

    #DERIV MINUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=wb_m, omch2=wc_m,omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    sig8_wb_m= results.get_sigma8()[0]
    #Pkz0_wb_m = interp1d(kh, pk[0])

    rt0_wb_m = interp1d(kh,pk[0]/pow(sig8_wb_m,2))

    #RESET FIDUCIAL
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)

    #print sig8_wb_m, sig8_fid, sig8_wb_p
    #print results.get_sigma8()[0]

    # Numerical derivative wrt wm 

    wm = (omegab+omegac)*pow(hubble,2)
    step_wm = 0.01*wm
    wm_p = wm+step_wm
    wm_m = wm-step_wm

    #DERIV PLUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=wb, omch2=wm_p-wb,omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    sig8_wm_p = results.get_sigma8()[0]
    #Pkz0_wm_p = interp1d(kh, pk[0])

    rt0_wm_p = interp1d(kh,pk[0]/pow(sig8_wm_p,2))

    #DERIV MINUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=wb, omch2=wm_m-wb,omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    sig8_wm_m= results.get_sigma8()[0]
    #Pkz0_wm_m = interp1d(kh, pk[0])

    rt0_wm_m = interp1d(kh,pk[0]/pow(sig8_wm_m,2))

    #RESET FIDUCIAL
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)

    #print sig8_wm_m, sig8_fid, sig8_wm_p
    #print results.get_sigma8()[0]

    # Numerical derivative wrt h 

    step_h = 0.01*hubble
    h_p = hubble+step_h
    h_m = hubble-step_h

    #DERIV PLUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=100*h_p, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    #print pars

    sig8_h_p = results.get_sigma8()[0]
    #Pkz0_h_p = interp1d(kh, pk[0])

    rt0_h_p = interp1d(kh,pk[0]/pow(sig8_h_p,2))

    #DERIV MINUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=100*h_m, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    #print pars

    sig8_h_m= results.get_sigma8()[0]
    #Pkz0_h_m = interp1d(kh, pk[0])

    rt0_h_m = interp1d(kh,pk[0]/pow(sig8_h_m,2))


    #RESET FIDUCIAL
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)

    print sig8_h_m, sig8_fid, sig8_h_p
    print results.get_sigma8()[0]

In [ ]:
# Shape parameter derivativers from scratch

# this bit never executes, but we are keeping it for the moment for comparisons
if 0:
    
    ############ Numerical derivative wrt ns

    #DERIV PLUS
    filename = pathrt+"Pks8sqRatio_ist_pl_ip=03_iz=00.dat"
    
    # Get the value of ns_p
    ns_p = 9.696e-01 # see Melita's file header            

    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_ns_p = interp1d(kh,rt, kind=interp_type)
        
    #DERIV MINUS
    filename = pathrt+"Pks8sqRatio_ist_mn_ip=03_iz=00.dat"
    
    # Get the value of ns_m
    ns_m = 9.504e-01 # see Melita's file header
    
    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_ns_m = interp1d(kh,rt, kind=interp_type)

    # Get the half-step
    step_ns = (ns_p - ns_m)/2
    

    ############ Numerical derivative wrt wb

    #DERIV PLUS
    filename = pathrt+"Pks8sqRatio_ist_pl_ip=01_iz=00.dat"
    
    # Get the value of wb_p
    wb_p = 2.26695e-02 # see Melita's file header            

    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_wb_p = interp1d(kh,rt, kind=interp_type)
    
    #DERIV MINUS
    filename = pathrt+"Pks8sqRatio_ist_mn_ip=01_iz=00.dat"
    
    # Get the value of wb_m
    wb_m = 2.22206e-02 # see Melita's file header
    
    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_wb_m = interp1d(kh,rt, kind=interp_type)

    # Get the half-step
    step_wb = (wb_p - wb_m)/2


    ############ Numerical derivative wrt wm

    #DERIV PLUS
    filename = pathrt+"Pks8sqRatio_ist_pl_ip=00_iz=00.dat"
    
    # Get the value of wm_p
    wm_p = 1.45084e-01 # see Melita's file header            

    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_wm_p = interp1d(kh,rt, kind=interp_type)
    
    #DERIV MINUS
    filename = pathrt+"Pks8sqRatio_ist_mn_ip=00_iz=00.dat"
    
    # Get the value of wm_m
    wm_m = 1.42212e-01 # see Melita's file header
    
    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_wm_m = interp1d(kh,rt, kind=interp_type)

    # Get the half-step
    step_wm = (wm_p - wm_m)/2


    ############ Numerical derivative wrt h

    #DERIV PLUS
    filename = pathrt+"Pks8sqRatio_ist_pl_ip=02_iz=00.dat"
    
    # Get the value of h_p
    h_p = 6.76700e-01 # see Melita's file header            

    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_h_p = interp1d(kh,rt, kind=interp_type)
    
    #DERIV MINUS
    filename = pathrt+"Pks8sqRatio_ist_mn_ip=02_iz=00.dat"
    
    # Get the value of h_m
    h_m = 6.63300e-01 # see Melita's file header
    
    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_h_m = interp1d(kh,rt, kind=interp_type) # THIS WAS A BUG!! MISSING THE INTERPOLATION TYPE!!

    # Get the half-step
    step_h = (h_p - h_m)/2
    

elif fromfile:
    
    ############ Numerical derivative wrt ns
    rt0_ns_p = {zc: ratio(zc,'pl','03') for zc in zlist} # DERIV PLUS
    rt0_ns_m = {zc: ratio(zc,'mn','03') for zc in zlist} # DERIV MINUS
    ns_p = 9.696e-01; ns_m = 9.504e-01                   # see Melita's file header
    step_ns = (ns_p - ns_m)/2                            # Get the half-step
    
    ############ Numerical derivative wrt wb
    rt0_wb_p = {zc: ratio(zc,'pl','01') for zc in zlist} # DERIV PLUS
    rt0_wb_m = {zc: ratio(zc,'mn','01') for zc in zlist} # DERIV MINUS
    wb_p = 2.26695e-02; wb_m = 2.22206e-02               # see Melita's file header
    step_wb = (wb_p - wb_m)/2                            # Get the half-step

    ############ Numerical derivative wrt wm
    rt0_wm_p = {zc: ratio(zc,'pl','00') for zc in zlist} # DERIV PLUS
    rt0_wm_m = {zc: ratio(zc,'mn','00') for zc in zlist} # DERIV MINUS
    wm_p = 1.45084e-01; wm_m = 1.42212e-01               # see Melita's file header
    step_wm = (wm_p - wm_m)/2                            # Get the half-step

    ############ Numerical derivative wrt h
    rt0_h_p = {zc: ratio(zc,'pl','02') for zc in zlist}  # DERIV PLUS
    rt0_h_m = {zc: ratio(zc,'mn','02') for zc in zlist}  # DERIV MINUS
    h_p = 6.76700e-01; h_m = 6.63300e-01                 # see Melita's file header
    step_h = (h_p - h_m)/2                               # Get the half-step

In [ ]:
# Get the derivatives wrt shape parameters
def dlnP_dns(kk,zc):
    if not fromfile:
        return (rt0_ns_p(kk)-rt0_ns_m(kk))/(2*step_ns)/rt0_fid(kk)
    else:
        return (rt0_ns_p[zc](kk)-rt0_ns_m[zc](kk))/(2*step_ns)/rt0_fid(kk)
    
def dlnP_dwb(kk,zc):
    if not fromfile:
        return (rt0_wb_p(kk)-rt0_wb_m(kk))/(2*step_wb)/rt0_fid(kk)
    else:
        return (rt0_wb_p[zc](kk)-rt0_wb_m[zc](kk))/(2*step_wb)/rt0_fid(kk)
    
def dlnP_dwm(kk,zc):
    if not fromfile:
        return (rt0_wm_p(kk)-rt0_wm_m(kk))/(2*step_wm)/rt0_fid(kk)
    else:
        return (rt0_wm_p[zc](kk)-rt0_wm_m[zc](kk))/(2*step_wm)/rt0_fid(kk)
    
def dlnP_dh(kk,zc):
    if not fromfile:
        return (rt0_h_p(kk)-rt0_h_m(kk))/(2*step_h)/rt0_fid(kk)
    else:
        return (rt0_h_p[zc](kk)-rt0_h_m[zc](kk))/(2*step_h)/rt0_fid(kk)
    
print dlnP_dns(0.1,0.7), dlnP_dwb(0.1,0.7), dlnP_dwm(0.1,0.7), dlnP_dh(0.1,0.7)
#    fromfile-linear: -0.458617307209 1.84905736371 -0.718184176150 -0.194940813117
#    fromfile -cubic: -0.458490904433 1.84082086107 -0.711493961367 -0.197117048004
# fromscratch-linear: -0.458732231721 1.83364997100 -0.696296384895 -0.223587693788
# fromscratch -cubic: -0.458732231721 1.83364997100 -0.696296384895 -0.223587693788

In [ ]:
# Auxiliary bits needed for Fisher matrix calculation

def dF(kk,mu,zc):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)  

# 2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.romb(dfun.T[i], dx=np.diff(mugrid)[0]) for i in range(kgrid.size)]
    return scipy.integrate.romb(muint, dx=np.diff(kgrid)[0])

mugrid = np.linspace(-1., 1., 513) 

In [ ]:
%%time
#################################################   Fisher matrix   #################################################

# Create array of zeros
Npar = len(params)
Npar_shape = len(shape_params)
s = (Npar-Npar_shape)*Nzbins + Npar_shape
Fishermat = np.zeros([s,s])

# Loop over redshift, k and mu and get all the Fisher matrix entries by calling the derivatives
kgrid = np.linspace(kmin, kmax, 513)
K, MU = np.meshgrid(kgrid, mugrid)
for zi in range(0,Nzbins):
    zc = zlist[zi]
    bg = biaslist[zi]
    zmin = zc-Dzbin/2
    zmax = zc+Dzbin/2
    Ngal = dn3[zi]*Area*Dzbin

    for i in range(0,Npar):
        if i not in shape_params: 
            k=zi*(Npar-Npar_shape) + i
        else:
            k = s + (i - Npar)
            
        def deriv_i(kk,mu,z):
            if i==0:  return dlnP_dlnH(kk,mu,z)
            elif i==1:  return dlnP_dlnDA(kk,mu,z)
            elif i==2:  return dlnP_dlnfsig8(kk,mu,z)
            elif i==3:  return dlnP_dlnbsig8(kk,mu,z)
            elif i==4:  return dlnP_dPs(kk,mu,z)
            elif i==5:  return dlnP_dns(kk,z)
            elif i==6:  return dlnP_dwb(kk,z)
            elif i==7:  return dlnP_dwm(kk,z)
            elif i==8:  return dlnP_dh(kk,z)
            else: print "Error: index out of range"
        
        for  j in range(0,Npar):
            if j not in shape_params:
                l=zi*(Npar-Npar_shape) + j
            else:
                l = s + (j - Npar)
                       
            if j>=i:
                def deriv_j(kk,mu,z):
                    if j==0:  return dlnP_dlnH(kk,mu,z)
                    elif j==1:  return dlnP_dlnDA(kk,mu,z)
                    elif j==2:  return dlnP_dlnfsig8(kk,mu,z)
                    elif j==3:  return dlnP_dlnbsig8(kk,mu,z)
                    elif j==4:  return dlnP_dPs(kk,mu,z)
                    elif j==5:  return dlnP_dns(kk,z)
                    elif j==6:  return dlnP_dwb(kk,z)
                    elif j==7:  return dlnP_dwm(kk,z)
                    elif j==8:  return dlnP_dh(kk,z)
                    else: print "Error: index out of range" 
                               
            if l>=k: 
                Fishermat[k][l] += integrate2D(dF(K,MU,zc),kgrid,mugrid)
            else: 
                Fishermat[k,l] = Fishermat[l,k]                 

In [ ]:
# Save the resulting Fisher matrix into a file
header = 'lnH_0.7 lnDa_0.7 lnfs8_0.7 lnbs8_0.7 Ps_0.7 ' +\
         'lnH_0.8 lnDa_0.8 lnfs8_0.8 lnbs8_0.8 Ps_0.8 ' +\
         'lnH_0.9 lnDa_0.9 lnfs8_0.9 lnbs8_0.9 Ps_0.9 ' +\
         'lnH_1.0 lnDa_1.0 lnfs8_1.0 lnbs8_1.0 Ps_1.0 ' +\
         'lnH_1.1 lnDa_1.1 lnfs8_1.1 lnbs8_1.1 Ps_1.1 ' +\
         'lnH_1.2 lnDa_1.2 lnfs8_1.2 lnbs8_1.2 Ps_1.2 ' +\
         'lnH_1.3 lnDa_1.3 lnfs8_1.3 lnbs8_1.3 Ps_1.3 ' +\
         'lnH_1.4 lnDa_1.4 lnfs8_1.4 lnbs8_1.4 Ps_1.4 ' +\
         'lnH_1.5 lnDa_1.5 lnfs8_1.5 lnbs8_1.5 Ps_1.5 ' +\
         'lnH_1.6 lnDa_1.6 lnfs8_1.6 lnbs8_1.6 Ps_1.6 ' +\
         'lnH_1.7 lnDa_1.7 lnfs8_1.7 lnbs8_1.7 Ps_1.7 ' +\
         'lnH_1.8 lnDa_1.8 lnfs8_1.8 lnbs8_1.8 Ps_1.8 ' +\
         'lnH_1.9 lnDa_1.9 lnfs8_1.9 lnbs8_1.9 Ps_1.9 ' +\
         'lnH_2.0 lnDa_2.0 lnfs8_2.0 lnbs8_2.0 Ps_2.0 ' +\
         'ns wb wm h'

# Generate an output file name
newfile = 'outputs/alkistis-case5' + '_' + interp_type
if fromfile:
    newfile += '-with-files'
    
# Save file
np.savetxt(newfile+misc.get_timestamp()+'.txt', Fishermat, header=header)

# Print for testing purposes
print np.sqrt(np.diag(linalg.inv(Fishermat)))

In [ ]:
# Cubic with files:
#    CPU times: user 34min 38s, sys: 3min 17s, total: 37min 55s
#    Wall time: 13min 7s

# Linear with files:
#    CPU times: user 1min 18s, sys: 904 ms, total: 1min 19s
#    Wall time: 1min 20s

# Cubic from scratch
#    CPU times: user 1min 17s, sys: 6.24 s, total: 1min 23s
#    Wall time: 1min 23s

# Linear from scratch
#    CPU times: user 1min 14s, sys: 5.86 s, total: 1min 20s
#    Wall time: 1min 20s